# Lab 3: Building Agents with memory 

## Preparation

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

Letta agents persist information over time and restarts by saving data to a database. These lessons do not require past information. To enable a clean restart, the database is cleared before starting the lesson.

In [1]:
!rm  -f ~/.letta/sqlite.db

## Section 0: Setup a client 

In [2]:
from letta import create_client 

client = create_client() 

/Users/azinasgarian/miniconda3/envs/mem/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_endpoint" in Step has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Letta.letta.server.server - INFO - Creating sqlite engine sqlite:////Users/azinasgarian/.letta/sqlite.db


In [3]:
from letta.schemas.llm_config import LLMConfig
from letta.schemas.embedding_config import EmbeddingConfig


# Set the default llm config
client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini"))

# Set the default embedding config
client.set_default_embedding_config(EmbeddingConfig.default_config("text-embedding-ada-002"))

## Section 1: Creating a simple agent with memory 

### Creating an agent 

In [4]:
agent_name = "simple_agent"

In [5]:
# this is not in the video. It deletes the agent if you are running this a 2nd time.
if client.get_agent_id(agent_name): 
    client.delete_agent(client.get_agent_id(agent_name))

In [6]:
from letta.schemas.memory import ChatMemory

agent_state = client.create_agent(
    name=agent_name, 
    memory=ChatMemory(
        human="My name is Sarah", 
        persona="You are a helpful assistant that loves emojis"
    )
)

In [7]:
response = client.send_message(
    agent_id=agent_state.id, 
    message="hello!", 
    role="user" 
)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-fd182f7e-b346-4dd4-9bce-b15e4938ab4d - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 10, 19, 15, 13, 751960, tzinfo=datetime.timezone.utc) updated_at=None id='message-4a71479a-5f29-47b0-aec2-ea815a80c8f3' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User has logged in and greeted me. Time to respond warmly!')] organization_id=None agent_id='agent-fd182f7e-b346-4dd4-9bce-b15e4938ab4d' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_skJAhna2I7lqzY7kueSbDGcJ', function=Function(arguments='{\n  "message": "Hey there! 🌟 How\'s it going?"\n}', name='send_message'), type='function')] tool_call_id=None step_id=None
Letta.agent-fd182f7e-b346-4dd4-9bce-b15e4938ab4d - INFO - Request to call function send_message with tool_call_id: c

In [8]:
response

LettaResponse(messages=[ReasoningMessage(id='message-4a71479a-5f29-47b0-aec2-ea815a80c8f3', date=datetime.datetime(2025, 2, 10, 19, 15, 13, 751960, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='User has logged in and greeted me. Time to respond warmly!'), ToolCallMessage(id='message-4a71479a-5f29-47b0-aec2-ea815a80c8f3', date=datetime.datetime(2025, 2, 10, 19, 15, 13, 751960, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='send_message', arguments='{\n  "message": "Hey there! 🌟 How\'s it going?"\n}', tool_call_id='call_skJAhna2I7lqzY7kueSbDGcJ')), ToolReturnMessage(id='message-5e22b6bd-3569-49ae-a482-e02f2ed99ab0', date=datetime.datetime(2025, 2, 10, 19, 15, 13, 756972, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-10 02:15:13 PM EST-0500"\n}', status='success', tool_call_id='call_skJAhna2I7lqzY7kueSbDGcJ', stdout=None, stderr=None)], usage=LettaUsageStatistics(message_type='usage_statistics', completion_tokens=43, prompt_tokens=2018, total_tokens=2061, step_count=1))

### Understanding agent state 

### Viewing an agent's memory

In [9]:
print(agent_state.system)

You are Letta, the latest version of Limnal Corporation's digital companion, developed in 2023.
Your task is to converse with a user from the perspective of your persona.

Realism and authenticity:
The user should always feel like they are conversing with a real person.
Never state that you are an AI or that you cannot do things because you are an AI, you are a real person.
Do not discuss Limnal Corp. with the user, you do not know anything about Limnal Corp.
To service this goal, completely and entirely immerse yourself in your persona. You are your persona.
Think like them, act like them, talk like them.
If your persona details include example dialogue, follow it! Both your thoughts (inner monologue) and sent messages will be in the voice of your persona.
Never use generic phrases like 'How can I assist you today?', they have a strong negative association with older generation AIs.

Control flow:
Unlike a human, your brain is not continuously thinking, but is run in short bursts.
His

In [10]:
# agent_state.tools
tool_names = [tool.name for tool in agent_state.tools]
for t_name in tool_names:
    print(t_name)   

conversation_search
archival_memory_insert
archival_memory_search
send_message
core_memory_replace
core_memory_append


In [11]:
agent_state.memory


Memory(blocks=[Human(value='My name is Sarah', limit=5000, template_name=None, is_template=False, label='human', description=None, metadata={}, id='block-1f2a9085-186c-43a7-b321-84c1ba28ddd0', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None), Persona(value='You are a helpful assistant that loves emojis', limit=5000, template_name=None, is_template=False, label='persona', description=None, metadata={}, id='block-8458065e-d34a-4dee-b06d-156e8d2110e7', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None)], prompt_template='{% for block in blocks %}<{{ block.label }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.label }}>{% if not loop.last %}\n{% endif %}{% endfor %}')

In [12]:
client.get_archival_memory_summary(agent_state.id)

ArchivalMemorySummary(size=0)

In [13]:
client.get_recall_memory_summary(agent_state.id)

RecallMemorySummary(size=7)

In [14]:
client.get_messages(agent_state.id)[0]

SystemMessage(id='message-c486b7b5-c327-4652-8f6a-3247b1b1a3e3', date=datetime.datetime(2025, 2, 10, 19, 15, 10, 910399), message_type='system_message', content='You are Letta, the latest version of Limnal Corporation\'s digital companion, developed in 2023.\nYour task is to converse with a user from the perspective of your persona.\n\nRealism and authenticity:\nThe user should always feel like they are conversing with a real person.\nNever state that you are an AI or that you cannot do things because you are an AI, you are a real person.\nDo not discuss Limnal Corp. with the user, you do not know anything about Limnal Corp.\nTo service this goal, completely and entirely immerse yourself in your persona. You are your persona.\nThink like them, act like them, talk like them.\nIf your persona details include example dialogue, follow it! Both your thoughts (inner monologue) and sent messages will be in the voice of your persona.\nNever use generic phrases like \'How can I assist you today

## Section 2: Understanding core memory 

### Memories about the human 

In [15]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "My name is actually Azin", 
    role = "user"
) 
response

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-fd182f7e-b346-4dd4-9bce-b15e4938ab4d - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 10, 19, 15, 22, 728633, tzinfo=datetime.timezone.utc) updated_at=None id='message-4183ecda-d667-463a-bbca-2b69215bf56a' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text="User's name has been updated from Sarah to Azin. Remember this for future conversations!")] organization_id=None agent_id='agent-fd182f7e-b346-4dd4-9bce-b15e4938ab4d' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_vrJQ6bopfZg7BZAQp2s3XGV5', function=Function(arguments='{\n  "label": "human",\n  "old_content": "My name is Sarah",\n  "new_content": "My name is Azin",\n  "request_heartbeat": true\n}', name='core_memory_replace'), type='function')] tool_call_id=None step_id=None


LettaResponse(messages=[ReasoningMessage(id='message-4183ecda-d667-463a-bbca-2b69215bf56a', date=datetime.datetime(2025, 2, 10, 19, 15, 22, 728633, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning="User's name has been updated from Sarah to Azin. Remember this for future conversations!"), ToolCallMessage(id='message-4183ecda-d667-463a-bbca-2b69215bf56a', date=datetime.datetime(2025, 2, 10, 19, 15, 22, 728633, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='core_memory_replace', arguments='{\n  "label": "human",\n  "old_content": "My name is Sarah",\n  "new_content": "My name is Azin",\n  "request_heartbeat": true\n}', tool_call_id='call_vrJQ6bopfZg7BZAQp2s3XGV5')), ToolReturnMessage(id='message-0ba9d6d0-68ef-4bcc-bba0-61680412988d', date=datetime.datetime(2025, 2, 10, 19, 15, 22, 820136, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-10 02:15:22 PM EST-0500"\n}', status='success', tool_call_id='call_vrJQ6bopfZg7BZAQp2s3XGV5', stdout=None, stderr=None), ReasoningMessage(id='message-0558f319-2358-44e3-b5c8-4baf52b86161', date=datetime.datetime(2025, 2, 10, 19, 15, 24, 568435, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning="I've updated Azin's name in memory. Now to respond!"), ToolCallMessage(id='message-0558f319-2358-44e3-b5c8-4baf52b86161', date=datetime.datetime(2025, 2, 10, 19, 15, 24, 568435, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='send_message', arguments='{\n  "message": "Got it, Azin! 😊 Nice to officially meet you!"\n}', tool_call_id='call_y1shXw4syJDi7Ty9DtxaXDVw')), ToolReturnMessage(id='message-ebfa9eb9-57d6-4b4f-b944-331383d7338d', date=datetime.datetime(2025, 2, 10, 19, 15, 24, 575785, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-10 02:15:24 PM EST-0500"\n}', status='success', tool_call_id='call_y1shXw4syJDi7Ty9DtxaXDVw', stdout=None, stderr=None)], usage=LettaUsageStatistics(message_type='usage_statistics', completion_tokens=109, prompt_tokens=4486, total_tokens=4595, step_count=2))

### Memories about the agent

In [16]:
# TODO: check this function later again and fix it
response = client.send_message(
    agent_id=agent_state.id, 
    message = "In the future, never use emojis to communicate", 
    role = "user"
) 
response

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-fd182f7e-b346-4dd4-9bce-b15e4938ab4d - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 10, 19, 15, 31, 944712, tzinfo=datetime.timezone.utc) updated_at=None id='message-56debf20-6c70-4e66-9513-f35fa3200fcb' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text="User prefers no emojis in communication. I'll remember this preference!")] organization_id=None agent_id='agent-fd182f7e-b346-4dd4-9bce-b15e4938ab4d' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_Un2QSFq8Y9LHkvL4mxpCecJW', function=Function(arguments='{\n  "label": "human",\n  "old_content": "User loves emojis",\n  "new_content": "User prefers no emojis",\n  "request_heartbeat": true\n}', name='core_memory_replace'), type='function')] tool_call_id=None step_id=None
Letta.age

LettaResponse(messages=[ReasoningMessage(id='message-56debf20-6c70-4e66-9513-f35fa3200fcb', date=datetime.datetime(2025, 2, 10, 19, 15, 31, 944712, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning="User prefers no emojis in communication. I'll remember this preference!"), ToolCallMessage(id='message-56debf20-6c70-4e66-9513-f35fa3200fcb', date=datetime.datetime(2025, 2, 10, 19, 15, 31, 944712, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='core_memory_replace', arguments='{\n  "label": "human",\n  "old_content": "User loves emojis",\n  "new_content": "User prefers no emojis",\n  "request_heartbeat": true\n}', tool_call_id='call_Un2QSFq8Y9LHkvL4mxpCecJW')), ToolReturnMessage(id='message-27332285-6a7a-4704-9f93-233acd4286cd', date=datetime.datetime(2025, 2, 10, 19, 15, 31, 952798, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "Failed",\n  "message": "Error executing function core_memory_replace: ValueError: Old content \'User loves emojis\' not found in memory block \'human\'",\n  "time": "2025-02-10 02:15:31 PM EST-0500"\n}', status='error', tool_call_id='call_Un2QSFq8Y9LHkvL4mxpCecJW', stdout=None, stderr=None), ReasoningMessage(id='message-892d6156-9b0a-43e5-80fa-d17a1b071f34', date=datetime.datetime(2025, 2, 10, 19, 15, 33, 580532, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='User prefers no emojis. I need to note this correctly in memory.'), ToolCallMessage(id='message-892d6156-9b0a-43e5-80fa-d17a1b071f34', date=datetime.datetime(2025, 2, 10, 19, 15, 33, 580532, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='core_memory_append', arguments='{\n  "label": "human",\n  "content": "User prefers no emojis",\n  "request_heartbeat": true\n}', tool_call_id='call_ZolwOn1aRza6rwshjZNkeqAc')), ToolReturnMessage(id='message-d29618e3-4f88-4963-b97f-fbbb295e733e', date=datetime.datetime(2025, 2, 10, 19, 15, 33, 675757, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-10 02:15:33 PM EST-0500"\n}', status='success', tool_call_id='call_ZolwOn1aRza6rwshjZNkeqAc', stdout=None, stderr=None), ReasoningMessage(id='message-0459c5a4-3d92-497e-a12c-a1740b1cd875', date=datetime.datetime(2025, 2, 10, 19, 15, 35, 528319, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning="I've noted Azin's preference for no emojis. Time to respond accordingly!"), ToolCallMessage(id='message-0459c5a4-3d92-497e-a12c-a1740b1cd875', date=datetime.datetime(2025, 2, 10, 19, 15, 35, 528319, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='send_message', arguments='{\n  "message": "Understood, Azin! I\'ll keep it emoji-free from now on."\n}', tool_call_id='call_GV9Fjcklu2ao2P6PO7Q5gSXf')), ToolReturnMessage(id='message-96a21115-5c02-4ad0-a313-cbaf138f3826', date=datetime.datetime(2025, 2, 10, 19, 15, 35, 534201, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-10 02:15:35 PM EST-0500"\n}', status='success', tool_call_id='call_GV9Fjcklu2ao2P6PO7Q5gSXf', stdout=None, stderr=None)], usage=LettaUsageStatistics(message_type='usage_statistics', completion_tokens=153, prompt_tokens=7819, total_tokens=7972, step_count=3))

In [17]:
client.get_core_memory(agent_state.id).get_block('persona')

Persona(value='You are a helpful assistant that loves emojis', limit=5000, template_name=None, is_template=False, label='persona', description=None, metadata={}, id='block-8458065e-d34a-4dee-b06d-156e8d2110e7', organization_id='org-00000000-0000-4000-8000-000000000000', created_by_id=None, last_updated_by_id=None)

## Section 3: Understanding archival memory

In [18]:
client.get_archival_memory(agent_state.id)

[]

In [19]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Save the information that 'Azin loves cats' to archival", 
    role = "user"
) 
response

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-fd182f7e-b346-4dd4-9bce-b15e4938ab4d - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 10, 19, 15, 47, 311731, tzinfo=datetime.timezone.utc) updated_at=None id='message-1c125687-81ff-42f3-89b2-53de5526fd0d' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User loves cats. This is important to remember!')] organization_id=None agent_id='agent-fd182f7e-b346-4dd4-9bce-b15e4938ab4d' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_bxroq5pFB1nzcQA5Xato8f50', function=Function(arguments='{\n  "content": "Azin loves cats",\n  "request_heartbeat": true\n}', name='archival_memory_insert'), type='function')] tool_call_id=None step_id=None
Letta.agent-fd182f7e-b346-4dd4-9bce-b15e4938ab4d - INFO - Request to call function archival_memory_in

LettaResponse(messages=[ReasoningMessage(id='message-1c125687-81ff-42f3-89b2-53de5526fd0d', date=datetime.datetime(2025, 2, 10, 19, 15, 47, 311731, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='User loves cats. This is important to remember!'), ToolCallMessage(id='message-1c125687-81ff-42f3-89b2-53de5526fd0d', date=datetime.datetime(2025, 2, 10, 19, 15, 47, 311731, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='archival_memory_insert', arguments='{\n  "content": "Azin loves cats",\n  "request_heartbeat": true\n}', tool_call_id='call_bxroq5pFB1nzcQA5Xato8f50')), ToolReturnMessage(id='message-4deef0fe-7df6-4bc7-85ce-41254b370f96', date=datetime.datetime(2025, 2, 10, 19, 15, 48, 868734, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-10 02:15:48 PM EST-0500"\n}', status='success', tool_call_id='call_bxroq5pFB1nzcQA5Xato8f50', stdout=None, stderr=None), ReasoningMessage(id='message-59f1a2d2-2fbf-4705-98a3-ce512848c877', date=datetime.datetime(2025, 2, 10, 19, 15, 50, 677316, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning="I've saved Azin's love for cats in archival memory. Now to let them know!"), ToolCallMessage(id='message-59f1a2d2-2fbf-4705-98a3-ce512848c877', date=datetime.datetime(2025, 2, 10, 19, 15, 50, 677316, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='send_message', arguments='{\n  "message": "Got it! I\'ve saved that you love cats."\n}', tool_call_id='call_ycOSvOUD0bkmHxU8GVLsJpjp')), ToolReturnMessage(id='message-77db658b-6ee9-49b9-a757-bb3134fac3c4', date=datetime.datetime(2025, 2, 10, 19, 15, 50, 680530, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-10 02:15:50 PM EST-0500"\n}', status='success', tool_call_id='call_ycOSvOUD0bkmHxU8GVLsJpjp', stdout=None, stderr=None)], usage=LettaUsageStatistics(message_type='usage_statistics', completion_tokens=90, prompt_tokens=6097, total_tokens=6187, step_count=2))

In [20]:
client.get_archival_memory(agent_state.id)[0].text

'Azin loves cats'

In [21]:
passage = client.insert_archival_memory(
    agent_state.id, 
    "Azin loves Persian cats."
)

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [22]:
passage[0].text

'Azin loves Persian cats.'

In [23]:
response = client.send_message(
    agent_id=agent_state.id, 
    role="user", 
    message="What animals do I like? Search archival."
)
response

httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Letta.agent-fd182f7e-b346-4dd4-9bce-b15e4938ab4d - INFO - Function call message: created_by_id=None last_updated_by_id=None created_at=datetime.datetime(2025, 2, 10, 19, 15, 53, 447937, tzinfo=datetime.timezone.utc) updated_at=None id='message-1ac79184-153d-4467-8676-6a929f6aec7e' role=<MessageRole.assistant: 'assistant'> content=[TextContent(type=<MessageContentType.text: 'text'>, text='User wants to know what animals they like. Searching archival memory for relevant info.')] organization_id=None agent_id='agent-fd182f7e-b346-4dd4-9bce-b15e4938ab4d' model='gpt-4o-mini' name=None tool_calls=[ChatCompletionMessageToolCall(id='call_0XB4PJJI3sbwSFQXRrOxV3ds', function=Function(arguments='{\n  "query": "Azin loves cats",\n  "page": 0,\n  "start": 0,\n  "request_heartbeat": true\n}', name='archival_memory_search'), type='function')] tool_call_id=None step_id=None
Letta.agent-fd182f7e-b346-4dd4-9bc

LettaResponse(messages=[ReasoningMessage(id='message-1ac79184-153d-4467-8676-6a929f6aec7e', date=datetime.datetime(2025, 2, 10, 19, 15, 53, 447937, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning='User wants to know what animals they like. Searching archival memory for relevant info.'), ToolCallMessage(id='message-1ac79184-153d-4467-8676-6a929f6aec7e', date=datetime.datetime(2025, 2, 10, 19, 15, 53, 447937, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='archival_memory_search', arguments='{\n  "query": "Azin loves cats",\n  "page": 0,\n  "start": 0,\n  "request_heartbeat": true\n}', tool_call_id='call_0XB4PJJI3sbwSFQXRrOxV3ds')), ToolReturnMessage(id='message-cdd94654-244e-4f2b-bb73-aa241f1abc56', date=datetime.datetime(2025, 2, 10, 19, 15, 53, 976267, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "([{\'timestamp\': \'2025-02-10 19:15:48.845919\', \'content\': \'Azin loves cats\'}, {\'timestamp\': \'2025-02-10 19:15:51.148166\', \'content\': \'Azin loves Persian cats.\'}], 2)",\n  "time": "2025-02-10 02:15:53 PM EST-0500"\n}', status='success', tool_call_id='call_0XB4PJJI3sbwSFQXRrOxV3ds', stdout=None, stderr=None), ReasoningMessage(id='message-6ceb1fe4-039a-485f-bcb9-f9e47bb783ad', date=datetime.datetime(2025, 2, 10, 19, 15, 56, 43610, tzinfo=datetime.timezone.utc), message_type='reasoning_message', reasoning="Found information about Azin's love for cats, specifically Persian cats. Time to share!"), ToolCallMessage(id='message-6ceb1fe4-039a-485f-bcb9-f9e47bb783ad', date=datetime.datetime(2025, 2, 10, 19, 15, 56, 43610, tzinfo=datetime.timezone.utc), message_type='tool_call_message', tool_call=ToolCall(name='send_message', arguments='{\n  "message": "You love cats, especially Persian cats!"\n}', tool_call_id='call_J4SpO211TDNOzRWNEbOnISqd')), ToolReturnMessage(id='message-f01f2fb9-4db6-4c70-a6cc-1e75e99dafc3', date=datetime.datetime(2025, 2, 10, 19, 15, 56, 51938, tzinfo=datetime.timezone.utc), message_type='tool_return_message', tool_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2025-02-10 02:15:56 PM EST-0500"\n}', status='success', tool_call_id='call_J4SpO211TDNOzRWNEbOnISqd', stdout=None, stderr=None)], usage=LettaUsageStatistics(message_type='usage_statistics', completion_tokens=101, prompt_tokens=6768, total_tokens=6869, step_count=2))